In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 설정 (선택사항)
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

print("=" * 60)
print("RANDOM FOREST REGRESSOR 모델")
print("=" * 60)

# 1. 데이터 로딩
print("\n[1단계] 데이터 로딩 중...")
df = pd.read_csv('Life Expectancy Data.csv')

print(f"데이터 크기: {df.shape}")
print(f"컬럼 수: {len(df.columns)}")

# 2. 데이터 전처리
print("\n[2단계] 데이터 전처리 중...")

# 결측치 확인
print(f"결측치 개수: {df.isnull().sum().sum()}개")

# 결측치를 중앙값으로 대체
df_cleaned = df.copy()
for col in df_cleaned.select_dtypes(include=[np.number]).columns:
    df_cleaned[col].fillna(df_cleaned[col].median(), inplace=True)

# 범주형 변수 인코딩
le_country = LabelEncoder()
le_status = LabelEncoder()
df_cleaned['Country'] = le_country.fit_transform(df_cleaned['Country'])
df_cleaned['Status'] = le_status.fit_transform(df_cleaned['Status'])

print(f"전처리 완료: 결측치 {df_cleaned.isnull().sum().sum()}개")

# 3. 특성(X)과 타겟(y) 분리
X = df_cleaned.drop(['Life expectancy '], axis=1)
y = df_cleaned['Life expectancy ']

print(f"\n[3단계] 데이터 분할")
print(f"특성(X): {X.shape}")
print(f"타겟(y): {y.shape}")

# 4. 학습/테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"학습 데이터: {X_train.shape}")
print(f"테스트 데이터: {X_test.shape}")

# 5. Random Forest 모델 학습
print("\n[4단계] Random Forest 모델 학습 중...")
print("하이퍼파라미터:")
print("  - n_estimators: 100 (트리 개수)")
print("  - max_depth: 15 (최대 깊이)")
print("  - random_state: 42")
print("  - n_jobs: -1 (모든 CPU 코어 사용)")

model = RandomForestRegressor(
    n_estimators=100,
    max_depth=15,
    random_state=42,
    n_jobs=-1
)

model.fit(X_train, y_train)

print("✓ 학습 완료!")

# 6. 예측
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# 7. 성능 평가
print("\n[5단계] 모델 성능 평가")
print("=" * 60)

train_mse = mean_squared_error(y_train, y_train_pred)
test_mse = mean_squared_error(y_test, y_test_pred)
train_rmse = np.sqrt(train_mse)
test_rmse = np.sqrt(test_mse)
train_mae = mean_absolute_error(y_train, y_train_pred)
test_mae = mean_absolute_error(y_test, y_test_pred)
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

print(f"{'평가 지표':<20} {'학습 데이터':<20} {'테스트 데이터':<20}")
print("-" * 60)
print(f"{'MSE':<20} {train_mse:<20.4f} {test_mse:<20.4f}")
print(f"{'RMSE':<20} {train_rmse:<20.4f} {test_rmse:<20.4f}")
print(f"{'MAE':<20} {train_mae:<20.4f} {test_mae:<20.4f}")
print(f"{'R² Score':<20} {train_r2:<20.4f} {test_r2:<20.4f}")

# 8. Feature Importance 분석
print("\n[6단계] 특성 중요도 분석 (Top 10)")
print("=" * 60)

feature_importance = pd.DataFrame({
    '특성': X.columns,
    '중요도': model.feature_importances_
}).sort_values('중요도', ascending=False)

print(feature_importance.head(10).to_string(index=False))

# 9. 시각화
print("\n[7단계] 시각화 생성 중...")

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 9-1. 실제값 vs 예측값
axes[0].scatter(y_test, y_test_pred, alpha=0.6, edgecolors='k', linewidth=0.5)
axes[0].plot([y_test.min(), y_test.max()], 
             [y_test.min(), y_test.max()], 
             'r--', lw=2, label='완벽한 예측선')
axes[0].set_xlabel('실제 수명 (년)', fontsize=11)
axes[0].set_ylabel('예측 수명 (년)', fontsize=11)
axes[0].set_title(f'Random Forest Regressor\nR² = {test_r2:.4f}, RMSE = {test_rmse:.4f}', 
                  fontsize=12, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 9-2. 잔차 플롯
residuals = y_test - y_test_pred
axes[1].scatter(y_test_pred, residuals, alpha=0.6, edgecolors='k', linewidth=0.5)
axes[1].axhline(y=0, color='r', linestyle='--', lw=2)
axes[1].set_xlabel('예측 수명 (년)', fontsize=11)
axes[1].set_ylabel('잔차 (실제값 - 예측값)', fontsize=11)
axes[1].set_title('잔차 플롯 (Residual Plot)', fontsize=12, fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
print("✓ [창 1] 예측 결과 및 잔차 플롯 표시")
plt.show()

# 10. Feature Importance 시각화
plt.figure(figsize=(10, 6))
top_features = feature_importance.head(10)
colors = plt.cm.viridis(np.linspace(0.3, 0.9, len(top_features)))
plt.barh(top_features['특성'], top_features['중요도'], color=colors)
plt.xlabel('특성 중요도', fontsize=11)
plt.title('상위 10개 특성 중요도 (Random Forest)', fontsize=12, fontweight='bold')
plt.gca().invert_yaxis()
plt.tight_layout()
print("✓ [창 2] Feature Importance 표시")
plt.show()

# 11. 트리별 예측 분포 시각화
plt.figure(figsize=(10, 6))
tree_predictions = []
for tree in model.estimators_[:10]:  # 처음 10개 트리만 시각화
    tree_pred = tree.predict(X_test)
    tree_predictions.append(tree_pred)
    plt.scatter(y_test, tree_pred, alpha=0.1, s=10)

plt.scatter(y_test, y_test_pred, alpha=0.8, s=30, c='red', 
            label='앙상블 예측 (평균)', edgecolors='k', linewidth=0.5)
plt.plot([y_test.min(), y_test.max()], 
         [y_test.min(), y_test.max()], 
         'b--', lw=2, label='완벽한 예측선')
plt.xlabel('실제 수명 (년)', fontsize=11)
plt.ylabel('예측 수명 (년)', fontsize=11)
plt.title('개별 트리 vs 앙상블 예측', fontsize=12, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
print("✓ [창 3] 앙상블 효과 시각화 표시")
plt.show()

# 12. 결과 저장
print("\n[8단계] 결과 저장 중...")

results = {
    '모델': 'Random Forest',
    'Test_RMSE': test_rmse,
    'Test_MAE': test_mae,
    'Test_R2': test_r2,
    'Train_RMSE': train_rmse,
    'Train_R2': train_r2,
    'n_estimators': 100,
    'max_depth': 15
}

results_df = pd.DataFrame([results])
results_df.to_csv('random_forest_metrics.csv', index=False)
print("✓ 성능 지표 저장: random_forest_metrics.csv")

# Feature Importance 저장
feature_importance.to_csv('random_forest_feature_importance.csv', index=False)
print("✓ Feature Importance 저장: random_forest_feature_importance.csv")

print("\n" + "=" * 60)
print("RANDOM FOREST 분석 완료!")
print("=" * 60)
print(f"\n최종 성능: R² = {test_r2:.4f}, RMSE = {test_rmse:.4f}")
print(f"가장 중요한 특성: {feature_importance.iloc[0]['특성']}")